In [35]:
import tensorflow as tf
import keras
import numpy as np
from typing import Tuple
from sklearn.svm import SVC


# Load the data and label
def load_data(filename: str) -> Tuple[np.ndarray, np.ndarray]:
    data = np.load(f'{filename}')
    return data['data'], data['labels']


# Load the data and labels
train_data, train_labels = load_data('train_data_SYN.npz')
test_data, test_labels = load_data('test_data_SYN.npz')

In [39]:
# Define the best model with L1 regularization and hidden layer of size 64
reg_l1_param = 10e-4
hidden_size = 64
input_shape = (1024,)
num_classes = 10

best_model = keras.models.Sequential([
    keras.layers.Dense(hidden_size, activation='relu', input_shape=input_shape, kernel_regularizer=keras.regularizers.l1(reg_l1_param)),
    keras.layers.Dense(num_classes, activation='softmax')
])
best_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train and evaluate the best model
best_model.fit(train_data, train_labels, epochs=10, batch_size=32, validation_data=(test_data, test_labels))
test_loss, test_acc = best_model.evaluate(test_data, test_labels)


weights = best_model.layers[0].get_weights()[0]
abs_weights = np.abs(weights)
feature_importance = np.sum(abs_weights, axis=1)


k = 110  #number of features to select
top_k_indices = np.argsort(feature_importance)[-k:]
top_k_data = train_data[:, top_k_indices]

# Train an SVM
svm = SVC(C=10,kernel="rbf")
svm.fit(top_k_data, train_labels)
svm_acc = svm.score(test_data[:, top_k_indices], test_labels)

# Calculate the score 
penalty_per_feature = 0.00075
penalties = k * penalty_per_feature
score = svm_acc - penalties
print(f'Score: accuracy({svm_acc}) - penalties({penalties}= )', score)

Epoch 1/10
157/157 [==============================] - 1s 4ms/step - loss: 2.8548 - accuracy: 0.7316 - val_loss: 2.1300 - val_accuracy: 0.8208
Epoch 2/10
157/157 [==============================] - 1s 3ms/step - loss: 1.5807 - accuracy: 0.8960 - val_loss: 1.4336 - val_accuracy: 0.8430
Epoch 3/10
157/157 [==============================] - 1s 4ms/step - loss: 1.0713 - accuracy: 0.9140 - val_loss: 1.1019 - val_accuracy: 0.8528
Epoch 4/10
157/157 [==============================] - 1s 3ms/step - loss: 0.8721 - accuracy: 0.9124 - val_loss: 0.9861 - val_accuracy: 0.8520
Epoch 5/10
157/157 [==============================] - 1s 4ms/step - loss: 0.7726 - accuracy: 0.9210 - val_loss: 0.9471 - val_accuracy: 0.8320
Epoch 6/10
157/157 [==============================] - 1s 3ms/step - loss: 0.7250 - accuracy: 0.9206 - val_loss: 0.8790 - val_accuracy: 0.8630
Epoch 7/10
157/157 [==============================] - 1s 4ms/step - loss: 0.6964 - accuracy: 0.9262 - val_loss: 0.8359 - val_accuracy: 0.8680
Epoch 